### Exploratory Data Analysis of Steam data

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [144]:
playtime_df = pd.read_csv('../data/steam_playtime.csv')

In [145]:
playtime_df.sort_values(by='playtime_forever', ascending=False).head()

,appid,steam_id,playtime_2weeks,playtime_forever
1390891,10,76561197995417670,19894,5096160
330644,550,76561198152959083,20154,3083602
330659,730,76561198152959083,20154,3007440
330671,218620,76561198152959083,20154,3002293
1293636,730,76561197960828778,0,2999875


In [146]:
len(playtime_df)

2731661

### User playtime data analysis
Play around with these to see which combinations yield better results in z-score, iqr and thereby the overall model

In [ ]:
# drop counterstrike i.e. drop rows where appid is 730, 10 or 240
playtime_df = playtime_df[~playtime_df['appid'].isin([730, 10, 240])]

In [147]:
# drop user playtimes over x minutes
playtime_df = playtime_df[playtime_df['playtime_forever'] < 1000000]

In [ ]:
# remove user playtimes under x minutes
playtime_df = playtime_df[playtime_df['playtime_forever'] > 60]

In [148]:
# list of users who have 2 week playtimes over x minutes
large_2week_playtimes = playtime_df[playtime_df['playtime_2weeks'] > 5000]

In [149]:
# drop users that are in large_2week_playtimes
playtime_df = playtime_df[~playtime_df['steam_id'].isin(large_2week_playtimes['steam_id'])]

In [ ]:
# get mean for playtime_forever grouped by steam_id
grouped_by_users_mean = playtime_df.groupby('steam_id')['playtime_forever'].mean()

In [150]:
# show biggest grouped_by_users_mean values in hours
grouped_by_users_mean.sort_values(ascending=False, ) / 60

steam_id
76561198414300864    28131.473958
76561198296559181    28034.989247
76561198282705580    28032.715054
76561198282489162    28030.566129
76561198282463381    28029.123656
                         ...     
76561197961574362        0.016667
76561198803446486        0.016667
76561198450047788        0.016667
76561198838743212        0.016667
76561198101758392        0.016667
Name: playtime_forever, Length: 21134, dtype: float64

In [ ]:
# drop extreme playtime_2weeks values, 5000 in 2 weeks equals 5.9 hours a day
playtime_df = playtime_df[playtime_df['playtime_2weeks'] < 5000]

### Z-SCORE

In [151]:
# get mean for playtime_forever
users_mean = playtime_df['playtime_forever'].mean()

In [152]:
# get standard deviation for playtime_forever
users_std = playtime_df['playtime_forever'].std()

In [153]:
print(users_mean, users_std)

2721.376776572513 20107.88125912218


In [154]:
def get_zscore(value, values):
    """Obtain the z-score of a given value"""
    m = users_mean
    s = users_std
    z_score = (value - m)/s
    return np.abs(z_score)

In [155]:
# compute z-scores for all values
playtime_df['z-score'] = playtime_df['playtime_forever'].apply(lambda x: get_zscore(x, playtime_df['playtime_forever']))

# find outliers
outliers = playtime_df[playtime_df['z-score']>3]
outliers.sort_values(by='playtime_forever', ascending=False)

,appid,steam_id,playtime_2weeks,playtime_forever,z-score
438761,444090,76561198277213142,0,975659,48.385885
23410,113400,76561198849564819,1455,975577,48.381807
23411,200210,76561198849564819,1455,975371,48.371562
23412,219340,76561198849564819,1455,975091,48.357637
23413,223710,76561198849564819,1455,974914,48.348835
...,...,...,...,...,...
1421692,730,76561197977695998,0,63053,3.000397
593809,635240,76561198107954601,0,63050,3.000248
593810,635241,76561198107954601,0,63048,3.000148
1459639,33930,76561198116089785,0,63048,3.000148


In [156]:
# z-score too big around the 131k mark
inliers = playtime_df[playtime_df['z-score']<3]
inliers.sort_values(by='playtime_forever', ascending=False)

,appid,steam_id,playtime_2weeks,playtime_forever,z-score
148705,240,76561197966939594,0,63044,2.999949
2529798,730,76561198324772456,0,63033,2.999402
28964,252490,76561198168997011,0,63033,2.999402
2192359,730,76561198287894651,0,63030,2.999253
2497400,286160,76561197978472660,0,63029,2.999203
...,...,...,...,...,...
867920,546350,76561197960568486,0,1,0.135289
1277517,479130,76561198123117316,0,1,0.135289
398929,219540,76561198042274496,0,1,0.135289
1277515,468560,76561198123117316,0,1,0.135289


In [157]:
print(inliers['playtime_forever'].mean(), inliers['playtime_forever'].std())

1328.2696859200717 4624.296032057817


In [158]:
# z-score does not take care of crazy playtime_2weeks values
inliers.sort_values(by=['playtime_2weeks'], ascending=False)

,appid,steam_id,playtime_2weeks,playtime_forever,z-score
1862309,578080,76561197960437463,4992,55049,2.602344
653989,271590,76561198143213523,4985,14525,0.587015
310923,1506830,76561198323712048,4985,14928,0.607057
2671844,730,76561198124963108,4983,11780,0.450501
2516891,1096900,76561197995213807,4977,4977,0.112176
...,...,...,...,...,...
920409,355000,76561198064815803,0,5,0.135090
920410,359320,76561198064815803,0,960,0.087596
920411,359550,76561198064815803,0,37558,1.732486
920412,623990,76561198064815803,0,708,0.100129


### Interquartile Range

In [159]:
# define the lower and upper bound 
q1 = np.quantile(playtime_df['playtime_forever'], 0.25)
q3 = np.quantile(playtime_df['playtime_forever'], 0.75) 
iqr = q3 - q1
lower_bound = q1 - 1.5*iqr
upper_bound = q3 + 1.5*iqr

# find records that fall outside of the lower and upper bound
iqr_outliers = playtime_df[(playtime_df['playtime_forever']<lower_bound) | (playtime_df['playtime_forever']>upper_bound)]
iqr_outliers.sort_values(by='playtime_forever', ascending=False)

,appid,steam_id,playtime_2weeks,playtime_forever,z-score
438761,444090,76561198277213142,0,975659,48.385885
23410,113400,76561198849564819,1455,975577,48.381807
23411,200210,76561198849564819,1455,975371,48.371562
23412,219340,76561198849564819,1455,975091,48.357637
23413,223710,76561198849564819,1455,974914,48.348835
...,...,...,...,...,...
2101396,6060,76561197976547922,0,1620,0.054773
2101389,7830,76561197976547922,0,1620,0.054773
846667,6060,76561197984817878,0,1620,0.054773
1073052,644930,76561197991886275,0,1620,0.054773


In [160]:
iqr_inliers = playtime_df[(playtime_df['playtime_forever']>lower_bound) & (playtime_df['playtime_forever']<upper_bound)]
iqr_inliers.sort_values(by='playtime_forever', ascending=False)

,appid,steam_id,playtime_2weeks,playtime_forever,z-score
97695,488460,76561198018420672,0,1618,0.054873
2351532,413150,76561197994622647,0,1618,0.054873
1705850,240,76561198101366739,0,1618,0.054873
2217542,273350,76561198045697930,0,1618,0.054873
123636,222880,76561198267883299,0,1618,0.054873
...,...,...,...,...,...
168188,1076280,76561198256081474,0,1,0.135289
1323311,453480,76561198320022297,0,1,0.135289
168184,1172470,76561198256081474,0,1,0.135289
2386929,35720,76561197987817692,0,1,0.135289


In [161]:
iqr_inliers.sort_values(by=['playtime_2weeks'], ascending=False)

,appid,steam_id,playtime_2weeks,playtime_forever,z-score
1134172,1149460,76561198178845671,1617,1617,0.054923
671479,1446780,76561198074913331,1616,1616,0.054972
1834688,1996040,76561197966826746,1612,1612,0.055171
1659055,1488200,76561198002604499,1608,1608,0.055370
1110410,1794680,76561197960452846,1606,1606,0.055470
...,...,...,...,...,...
907804,1324780,76561197971537009,0,33,0.133698
907803,1315980,76561197971537009,0,16,0.134543
907802,840390,76561197971537009,0,69,0.131907
907801,1151340,76561197971537009,0,639,0.103560


In [ ]:
iqr_inliers.to_csv('../data/steam_playtime_clean_iqr.csv', index=False)

### EXPLORATION & PLOTS

In [ ]:
# plot distribution of playtime_forever
sns.distplot(playtime_df['playtime_forever'], kde=False, bins=10)

In [ ]:
# histogram of playtime_forever
num_bins = 20
range_min, range_max = np.min(playtime_df['playtime_forever']), np.max(playtime_df['playtime_forever'])

plt.hist(playtime_df['playtime_forever'], bins=num_bins, range=(range_min, range_max))

# labels
plt.xlabel('Playtime (minutes)')
plt.ylabel('Frequency')
plt.title('Distribution of playtime')

# show plot
plt.show()

In [ ]:
sort_by_2weeks = playtime_df.sort_values(by='playtime_2weeks', ascending=False)

In [ ]:
# plot sort_by_2weeks
sns.distplot(sort_by_2weeks['playtime_2weeks'], kde=False, bins=10)


In [ ]:
# sort by playtime_forever
sort_by_playtime = playtime_df.sort_values(by = ['playtime_forever'], ascending = False).reset_index()

In [ ]:
# plot distribution of playtimes
sort_by_playtime.plot(kind = 'hist')

In [ ]:
sort_by_playtime

In [ ]:
sort_by_playtime.plot(y='playtime_forever', use_index=True)

In [ ]:
# plot sort_by_playtime
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.histplot(sort_by_playtime['playtime_forever'], bins = 20, kde = True)


In [ ]:
sns.boxplot(x=sort_by_playtime['playtime_forever'])
plt.show()

In [ ]:
# group by appid and get count, mean, median, min, max
playtime_by_game = playtime_df.groupby('appid').agg({'playtime_forever': ['count', 'mean', 'median', 'min', 'max']}).round(2)

In [ ]:
# sort by mean
playtime_by_game.sort_values(by=('playtime_forever', 'mean'), ascending=False)

In [ ]:
# sort by count
playtime_by_game.sort_values(by=('playtime_forever', 'count'), ascending=True)

In [ ]:
# playtime_by_game where count is 1
playtime_by_game[playtime_by_game[('playtime_forever', 'count')] == 1]

In [ ]:
# plot playtime by game
plt.figure(figsize=(20,10))
sns.barplot(x=playtime_by_game.index, y = playtime_by_game[('playtime_forever', 'mean')])
plt.title('Average Playtime by Game')
plt.xlabel('Game')


In [ ]:
# get appids where count is 1
only_one_playtime = playtime_by_game[playtime_by_game[('playtime_forever', 'count')] == 1].index

In [ ]:
# exclude games with only one playtime
playtime_df = playtime_df[~playtime_df['appid'].isin(only_one_playtime)]

In [ ]:
playtime_df

### GAME METADATA

In [2]:
games_df = pd.read_csv('../data/steam_app_metadata.csv')

In [3]:
print(len(games_df))

26180


In [4]:
games_df

,appid,name,description,developer,publisher,categories,genres
0,240,Counter-Strike: Source,THE NEXT INSTALLMENT OF THE WORLD'S # 1 ONLINE...,Valve,Valve,"['Multi-player', 'Cross-Platform Multiplayer',...",['Action']
1,300,Day of Defeat: Source,Day of Defeat offers intense online action gam...,Valve,Valve,"['Multi-player', 'Cross-Platform Multiplayer',...",['Action']
2,320,Half-Life 2: Deathmatch,Fast multiplayer action set in the Half-Life 2...,Valve,Valve,"['Multi-player', 'Valve Anti-Cheat enabled', '...",['Action']
3,340,Half-Life 2: Lost Coast,Originally planned as a section of the Highway...,Valve,Valve,"['Single-player', 'Commentary available', 'Rem...",['Action']
4,10180,Call of Duty®: Modern Warfare® 2,The most-anticipated game of the year and the ...,Infinity Ward,Activision,"['Single-player', 'Multi-player', 'Co-op', 'St...",['Action']
...,...,...,...,...,...,...,...
26175,626760,VRtender,In VRtender you'll play through several nights...,Further Beyond Studios,Further Beyond Studios,"['Single-player', 'Steam Achievements', 'Track...","['Action', 'Casual', 'Indie', 'Simulation']"
26176,2242760,The Escape: Together,<strong>The Escape: Together</strong> is a 1-3...,Nightstuff Games,Nightstuff Games,"['Single-player', 'Multi-player', 'Co-op', 'On...","['Action', 'Indie', 'Early Access']"
26177,2095300,Breakout 13,<h1>FOLLOW US</h1><p>More about the game. Plea...,ALT Lab,ALT Lab,"['Single-player', 'Steam Achievements', 'In-Ap...","['Adventure', 'Indie', 'RPG']"
26178,2094320,Escape First Alchemist ⚗️,Escape first: Alchemist is a multiplayer first...,OnSkull Games,OnSkull Games,"['Single-player', 'Multi-player', 'Co-op', 'On...","['Action', 'Adventure', 'Casual', 'Indie', 'Si..."


In [ ]:
# show duplicates for appid
games_df[games_df.duplicated(subset=['appid'])]

In [ ]:
# remove duplicate appid
games_df = games_df.drop_duplicates(subset=['appid'])

In [ ]:
# get duplicates for name
dupl_game_names = games_df[games_df.duplicated(subset=['name'])]

In [ ]:
# rename appid's in users_df that are duplicates to original appid
for index, row in dupl_game_names.iterrows():
    dupl_appid = row['appid']
    orig_appid = games_df[games_df['name'] == row['name']]['appid'].iloc[0]
    playtime_df.loc[playtime_df['appid'] == dupl_appid, 'appid'] = orig_appid


In [ ]:
# remove duplicates from games_df
games_df = games_df.drop_duplicates(subset=['name'])

In [ ]:
# count nulls for each column
games_df.isnull().sum()

In [ ]:
no_description = games_df[games_df['description'].isnull() | (games_df['description'] == '')]
games_df = games_df[~games_df['appid'].isin(no_description['appid'])]

In [ ]:
no_dev_or_pub = games_df[(games_df['developer'] == 'None') & games_df['publisher'].isnull()]
games_df = games_df[~games_df['appid'].isin(no_dev_or_pub['appid'])]

In [ ]:
# impute developer into publisher where publisher is null
games_df['publisher'] = games_df['publisher'].fillna(games_df['developer'])

In [5]:
# get average 'description' length
print('mean is', games_df['description'].str.len().mean().__round__(), 'characters')
print('median is', games_df['description'].str.len().median(), 'characters')

mean is 1993 characters
median is 1626.0 characters


In [ ]:
# get long length descriptions
games_df[games_df['description'].str.len() > 10000]